In [4]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
import lightgbm as lgb

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate
# from sklearn.inspection import permutation_importance

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [5]:
lines = ['T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305']


def seperate_code():
    data_path = '/Users/sola/Downloads/open/'
    output_dir = "Input/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    df = pd.read_csv(data_path + 'train.csv')

    for line in lines:
        df_line = df[df["LINE"] == line]
        df_line = df_line.dropna(how="any", axis="columns")

        # 모든 값이 동일한 열 제거
        cols = df_line.columns[6:]
        for col in cols:
            if len(df_line[col].unique()) == 1:
                df_line = df_line.drop([col], axis=1)

        df_line.to_csv(os.path.join(output_dir, f"{line}.csv"), index=False)


seperate_code()

In [6]:
def find_duplicate_col():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')
    df = df.round(0)

    drop_cols = []
    for line in lines:
        df_line = df[df["LINE"] == line]
        df_line = df_line.dropna(how="any", axis="columns")

        df_line = df_line.loc[:, ~df.T.duplicated()]

        cols = df_line.columns[6:]
        for col in cols:
            if len(df_line[col].unique()) == 1:
                df_line = df_line.drop([col], axis=1)

        drop_cols.append(df_line.columns)

    return drop_cols


find_duplicate_col()

[Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_128', 'X_129', 'X_136', 'X_137',
        ...
        'X_2796', 'X_2797', 'X_2798', 'X_2799', 'X_2800', 'X_2801', 'X_2837',
        'X_2839', 'X_2840', 'X_2841'],
       dtype='object', length=301),
 Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_130', 'X_136', 'X_137', 'X_138',
        ...
        'X_2796', 'X_2797', 'X_2798', 'X_2799', 'X_2800', 'X_2801', 'X_2837',
        'X_2839', 'X_2840', 'X_2841'],
       dtype='object', length=630),
 Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_1', 'X_2', 'X_5', 'X_11',
        ...
        'X_773', 'X_774', 'X_775', 'X_883', 'X_884', 'X_885', 'X_898', 'X_899',
        'X_900', 'X_901'],
       dtype='object', length=120),
 Index(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP', 'LINE',
        'PRODUCT_CODE', 'X_1', 'X_2', 'X_7', 'X_8',
        ...
        'X

In [7]:
def make_col():
    col_list = []
    for l, line in enumerate(lines):
        df = pd.read_csv(f"Input/{line}.csv")

        X = df[find_duplicate_col()[l]].drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])

        X.to_csv(f"Input/{line}.csv", index=False)
        col_list.append(X.columns)

    # Code : A
    A_col = list(set(col_list[0]) & set(col_list[1]) & set(col_list[4]) & set(col_list[5]))
    # Code : T, O
    OT_col = list(set(col_list[2]) & set(col_list[3]))

    use_col = A_col + OT_col

    return use_col, A_col, OT_col


make_col()

(['X_1099',
  'X_1109',
  'X_1112',
  'X_1527',
  'X_1140',
  'X_1104',
  'X_2050',
  'X_956',
  'X_1154',
  'X_1134',
  'X_1530',
  'X_1223',
  'X_1406',
  'X_1106',
  'X_1152',
  'X_1153',
  'X_1552',
  'X_1542',
  'X_1078',
  'X_1094',
  'X_1101',
  'X_1543',
  'X_1107',
  'X_1215',
  'X_1125',
  'X_1193',
  'X_1227',
  'X_1521',
  'X_1133',
  'X_1121',
  'X_1090',
  'X_1197',
  'X_1143',
  'X_1091',
  'X_1136',
  'X_1217',
  'X_1128',
  'X_1213',
  'X_1086',
  'X_2048',
  'X_1210',
  'X_1088',
  'X_1201',
  'X_1212',
  'X_1118',
  'X_1205',
  'X_1097',
  'X_1156',
  'X_1549',
  'X_1116',
  'X_1220',
  'X_1149',
  'X_2046',
  'X_1199',
  'X_1226',
  'X_1079',
  'X_1151',
  'X_1115',
  'X_1100',
  'X_1221',
  'X_1517',
  'X_1539',
  'X_1546',
  'X_1329',
  'X_1195',
  'X_2051',
  'X_1155',
  'X_1084',
  'X_1528',
  'X_1081',
  'X_1080',
  'X_1110',
  'X_1139',
  'X_1211',
  'X_1554',
  'X_1131',
  'X_1194',
  'X_1544',
  'X_1202',
  'X_1093',
  'X_1122',
  'X_1222',
  'X_2049',
  'X_

In [89]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


def data_modeling_A():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')

    data_A = df[df["PRODUCT_CODE"] == "A_31"]
    # data_A = data_A.fillna(0)
    A_y = data_A["Y_Class"]
    A_X = data_A.drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])

    X_train, X_test, y_train, y_test = train_test_split(A_X, A_y, test_size=0.3, random_state=42)

    model_A = lgb.LGBMClassifier(random_state=42)
    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    # result = model_A.feature_importances_
    # print(result.importances_mean)
    importances = model_A.feature_importances_
    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(X_train.shape[1]):
        if importances[indices[i]] > 0:
            a.append(feat_labels[indices[i]])
            # print(a)

    # a=[]
    # for i in range(100):
    #     a.append(feat_labels[indices[i]])
    # print(a)

    print(confusion_matrix(prediction, y_test))
    print(f"A_정답률:{accuracy_score(prediction, y_test):.3f}")

    print(a)

    return model_A, A_X.columns, a


data_modeling_A()

[[11  5  1]
 [ 5 29  5]
 [ 1  2 16]]
A_정답률:0.747
['X_318', 'X_367', 'X_1099', 'X_1430', 'X_368', 'X_1774', 'X_1295', 'X_287', 'X_1389', 'X_189', 'X_1053', 'X_1804', 'X_1743', 'X_1205', 'X_1197', 'X_1162', 'X_1954', 'X_1207', 'X_1560', 'X_953', 'X_959', 'X_1117', 'X_1948', 'X_248', 'X_1336', 'X_1202', 'X_1263', 'X_1949', 'X_286', 'X_258', 'X_1033', 'X_1048', 'X_1213', 'X_1376', 'X_1697', 'X_1120', 'X_1286', 'X_1108', 'X_1856', 'X_1744', 'X_1356', 'X_136', 'X_1290', 'X_345', 'X_1379', 'X_1243', 'X_999', 'X_1612', 'X_1665', 'X_1373', 'X_1390', 'X_1542', 'X_1833', 'X_1013', 'X_1109', 'X_1030', 'X_1058', 'X_1629', 'X_1643', 'X_1359', 'X_1155', 'X_1059', 'X_1428', 'X_1228', 'X_1262', 'X_1040', 'X_1231', 'X_1557', 'X_975', 'X_2050', 'X_985', 'X_1352', 'X_2063', 'X_1296', 'X_1329', 'X_976', 'X_339', 'X_977', 'X_1091', 'X_1735', 'X_2083', 'X_1124', 'X_1029', 'X_256', 'X_1176', 'X_1211', 'X_1556', 'X_1347', 'X_261', 'X_1371', 'X_993', 'X_979', 'X_1945', 'X_991', 'X_1408', 'X_1129', 'X_1636', 'X_

(LGBMClassifier(random_state=42),
 Index(['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10',
        ...
        'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
        'X_2873', 'X_2874', 'X_2875'],
       dtype='object', length=2875),
 ['X_318',
  'X_367',
  'X_1099',
  'X_1430',
  'X_368',
  'X_1774',
  'X_1295',
  'X_287',
  'X_1389',
  'X_189',
  'X_1053',
  'X_1804',
  'X_1743',
  'X_1205',
  'X_1197',
  'X_1162',
  'X_1954',
  'X_1207',
  'X_1560',
  'X_953',
  'X_959',
  'X_1117',
  'X_1948',
  'X_248',
  'X_1336',
  'X_1202',
  'X_1263',
  'X_1949',
  'X_286',
  'X_258',
  'X_1033',
  'X_1048',
  'X_1213',
  'X_1376',
  'X_1697',
  'X_1120',
  'X_1286',
  'X_1108',
  'X_1856',
  'X_1744',
  'X_1356',
  'X_136',
  'X_1290',
  'X_345',
  'X_1379',
  'X_1243',
  'X_999',
  'X_1612',
  'X_1665',
  'X_1373',
  'X_1390',
  'X_1542',
  'X_1833',
  'X_1013',
  'X_1109',
  'X_1030',
  'X_1058',
  'X_1629',
  'X_1643',
  'X_1359',
  'X_1155',
  

In [91]:
# 318한개로 돌려보기....?

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def data_modeling_Aa():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')

    data_A = df[df["PRODUCT_CODE"] == "A_31"]
    # data_A = data_A.fillna(0)
    A_y = data_A["Y_Class"]
    A_X = data_A.drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])
    A_X = A_X[['X_318',
               'X_367',
               'X_1099',
               'X_1430',
               'X_368',
               'X_1774',
               'X_1295',
               'X_287',
               'X_1389',
               'X_189',
               'X_1053',
               'X_1804',
               'X_1743',
               'X_1205',
               'X_1197',
               'X_1162',
               'X_1954',
               'X_1207',
               'X_1560',
               'X_953',
               'X_959',
               'X_1117',
               'X_1948',
               'X_248',
               'X_1336',
               'X_1202',
               'X_1263',
               'X_1949',
               'X_286',
               'X_258',
               'X_1033',
               'X_1048',
               'X_1213',
               'X_1376',
               'X_1697',
               'X_1120',
               'X_1286',
               'X_1108',
               'X_1856',
               'X_1744',
               'X_1356',
               'X_136',
               'X_1290',
               'X_345',
               'X_1379',
               'X_1243',
               'X_999',
               'X_1612',
               'X_1665',
               'X_1373',
               'X_1390',
               'X_1542',
               'X_1833',
               'X_1013',
               'X_1109',
               'X_1030',
               'X_1058',
               'X_1629',
               'X_1643',
               'X_1359',
               'X_1155',
               'X_1059',
               'X_1428',
               'X_1228',
               'X_1262',
               'X_1040',
               'X_1231',
               'X_1557',
               'X_975',
               'X_2050',
               'X_985',
               'X_1352',
               'X_2063',
               'X_1296',
               'X_1329',
               'X_976',
               'X_339',
               'X_977',
               'X_1091',
               'X_1735',
               'X_2083',
               'X_1124',
               'X_1029',
               'X_256',
               'X_1176',
               'X_1211',
               'X_1556',
               'X_1347',
               'X_261',
               'X_1371',
               'X_993',
               'X_979',
               'X_1945',
               'X_991',
               'X_1408',
               'X_1129',
               'X_1636',
               'X_2779',
               'X_954',
               'X_1564',
               'X_1806',
               'X_1967',
               'X_271',
               'X_2423',
               'X_1621',
               'X_949',
               'X_1374',
               'X_1272',
               'X_1191',
               'X_1183',
               'X_247',
               'X_1532',
               'X_1380',
               'X_955',
               'X_1366',
               'X_349',
               'X_1951',
               'X_2841',
               'X_1265',
               'X_1849',
               'X_373',
               'X_1289',
               'X_2791',
               'X_1287',
               'X_996',
               'X_1720',
               'X_2418',
               'X_1044',
               'X_1423',
               'X_1543',
               'X_1787',
               'X_1351',
               'X_1075',
               'X_2434',
               'X_1260',
               'X_1538',
               'X_1212',
               'X_298',
               'X_242',
               'X_1055',
               'X_1054',
               'X_1089',
               'X_1414',
               'X_1169',
               'X_1061',
               'X_1793',
               'X_1518',
               'X_986',
               'X_1038',
               'X_1215',
               'X_1866',
               'X_957',
               'X_1276',
               'X_973',
               'X_1966',
               'X_353',
               'X_1850',
               'X_1340',
               'X_1616',
               'X_2415',
               'X_1214',
               'X_967',
               'X_1387',
               'X_1609',
               'X_1140',
               'X_2097',
               'X_1625',
               'X_307',
               'X_285',
               'X_1427',
               'X_2051',
               'X_1569',
               'X_1111',
               'X_1280',
               'X_1548',
               'X_1100',
               'X_1381',
               'X_1274',
               'X_1348',
               'X_1098',
               'X_944',
               'X_1690',
               'X_1139',
               'X_961',
               'X_994',
               'X_1078',
               'X_1256',
               'X_1128',
               'X_1955',
               'X_146',
               'X_2190',
               'X_1521',
               'X_1045',
               'X_1651',
               'X_1258',
               'X_1194',
               'X_1028',
               'X_1584',
               'X_1177',
               'X_1018',
               'X_1563',
               'X_1854',
               'X_960',
               'X_1587',
               'X_1182',
               'X_1818',
               'X_257',
               'X_995',
               'X_1110',
               'X_380',
               'X_364',
               'X_1106',
               'X_1669',
               'X_1816',
               'X_1121',
               'X_1125',
               'X_1126',
               'X_1088',
               'X_1082',
               'X_1077',
               'X_1718',
               'X_1650',
               'X_964',
               'X_1135',
               'X_1638',
               'X_1072',
               'X_1633',
               'X_948',
               'X_310',
               'X_304',
               'X_1153',
               'X_1611',
               'X_1154',
               'X_1167',
               'X_267',
               'X_1740',
               'X_1064',
               'X_1862',
               'X_1227',
               'X_2747',
               'X_1369',
               'X_1370',
               'X_1292',
               'X_1223',
               'X_1208',
               'X_1257',
               'X_1424',
               'X_2787',
               'X_1534',
               'X_2041',
               'X_1291',
               'X_1242',
               'X_1526',
               'X_2753',
               'X_1337',
               'X_1338',
               'X_1350',
               'X_1341',
               'X_2793',
               'X_1552',
               'X_1551',
               'X_1421',
               'X_1418',
               'X_1349',
               'X_2044',
               'X_1353',
               'X_1275',
               'X_1297',
               'X_2788',
               'X_1358',
               'X_1204',
               'X_2771',
               'X_1708',
               'X_1076',
               'X_2437',
               'X_1554',
               'X_1716',
               'X_1429',
               'X_2430',
               'X_2782',
               'X_1301',
               'X_1422',
               'X_1063',
               'X_2412',
               'X_2410',
               'X_1083',
               'X_379',
               'X_1694',
               'X_1114',
               'X_350',
               'X_354',
               'X_1273',
               'X_356',
               'X_1118',
               'X_1277',
               'X_1278',
               'X_1279',
               'X_365',
               'X_366',
               'X_370',
               'X_1097',
               'X_1113',
               'X_2750',
               'X_1342',
               'X_1107',
               'X_1420',
               'X_1288',
               'X_1679',
               'X_1104',
               'X_1101',
               'X_1686',
               'X_1736',
               'X_1051',
               'X_2792',
               'X_945',
               'X_2078',
               'X_939',
               'X_1391',
               'X_1388',
               'X_2061',
               'X_1821',
               'X_1822',
               'X_1383',
               'X_984',
               'X_982',
               'X_981',
               'X_1832',
               'X_2794',
               'X_1375',
               'X_2843',
               'X_1372',
               'X_1339',
               'X_1357',
               'X_1355',
               'X_1354',
               'X_1346',
               'X_1858',
               'X_1345',
               'X_1860',
               'X_2058',
               'X_2082',
               'X_1000',
               'X_1398',
               'X_1419',
               'X_1417',
               'X_1416',
               'X_1052',
               'X_1748',
               'X_1123',
               'X_1415',
               'X_1047',
               'X_2800',
               'X_2801',
               'X_1757',
               'X_1043',
               'X_1042',
               'X_2096',
               'X_1036',
               'X_1035',
               'X_2047',
               'X_1406',
               'X_1026',
               'X_1780',
               'X_1019',
               'X_1016',
               'X_1002',
               'X_1660',
               'X_1344',
               'X_1588',
               'X_1171',
               'X_2019',
               'X_1185',
               'X_1536',
               'X_1217',
               'X_1630',
               'X_1246',
               'X_1218',
               'X_2701',
               'X_1145',
               'X_1174',
               'X_306',
               'X_1517',
               'X_1148',
               'X_2714',
               'X_1963',
               'X_1618',
               'X_1617',
               'X_1615',
               'X_1170',
               'X_265',
               'X_1232',
               'X_266',
               'X_1156',
               'X_1226',
               'X_1225',
               'X_1168',
               'X_2020',
               'X_1224',
               'X_2715',
               'X_1189',
               'X_1190',
               'X_142',
               'X_1561',
               'X_338',
               'X_129',
               'X_1657',
               'X_344',
               'X_2023',
               'X_140',
               'X_1553',
               'X_1544',
               'X_1646',
               'X_138',
               'X_1187',
               'X_1133',
               'X_1200',
               'X_1188',
               'X_2731']]

    X_train, X_test, y_train, y_test = train_test_split(A_X, A_y, test_size=0.3, random_state=42)

    model_A = lgb.LGBMClassifier(random_state=42)
    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    model_A.fit(X_train, y_train)
    prediction = model_A.predict(X_test)

    print(confusion_matrix(prediction, y_test))
    print(f"A_정답률:{accuracy_score(prediction, y_test):.3f}")

    return model_A, A_X.columns


data_modeling_Aa()

[[10  4  1]
 [ 6 28  5]
 [ 1  4 16]]
A_정답률:0.720


(LGBMClassifier(random_state=42),
 Index(['X_318', 'X_367', 'X_1099', 'X_1430', 'X_368', 'X_1774', 'X_1295',
        'X_287', 'X_1389', 'X_189',
        ...
        'X_140', 'X_1553', 'X_1544', 'X_1646', 'X_138', 'X_1187', 'X_1133',
        'X_1200', 'X_1188', 'X_2731'],
       dtype='object', length=418))

In [46]:
# from sklearn.ensemble import RandomForestClassifier
#
# feat_labels = X_train.columns
#
# rf = RandomForestClassifier(n_estimators = 1000, random_state = 0, n_jobs = -1)
#
# rf.fit(X_train, y_train)
# importances = rf.feature_importances_
#
# # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
# indices = np.argsort(importances)[::-1]
#
# for i in range(X_train.shape[1]):
# 	print(feat_labels[indices[i]], importances[indices[i]])
#
# #상위20개만...
# # a=[]
# # for i in range(80):
# #     a.append(feat_labels[indices[i]])
# # print(a)

In [8]:
from sklearn.ensemble import RandomForestClassifier

seperate_code()


def data_modeling_T():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')
    data_T = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]

    lists = []
    for a in make_col()[2]:
        lists.append(int(a[2:]))
    lists.sort()

    cols = []  # 사용할 columns
    for l in lists:
        cols.append(f"X_{l}")

    T_y = data_T["Y_Class"]
    T_X = data_T[cols]

    X_train, X_test, y_train, y_test = train_test_split(T_X, T_y, test_size=0.3, random_state=42)

    model_T = lgb.LGBMClassifier(random_state=42)
    model_T.fit(X_train, y_train)
    prediction = model_T.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    importances = model_T.feature_importances_

    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(X_train.shape[1]):
        if importances[indices[i]] > 1:
            a.append(feat_labels[indices[i]])

    print(confusion_matrix(prediction, y_test))
    print(f"T_정답률:{accuracy_score(prediction, y_test):.3f}")
    print(a)
    return model_T, cols


data_modeling_T()

[[ 4  1  0]
 [ 5 81 13]
 [ 0  1  0]]
T_정답률:0.810
['X_454', 'X_120', 'X_456', 'X_401', 'X_899', 'X_536', 'X_757', 'X_772', 'X_461', 'X_775', 'X_458', 'X_754', 'X_460', 'X_698', 'X_766', 'X_901', 'X_446', 'X_453', 'X_763', 'X_562', 'X_457', 'X_452', 'X_2', 'X_769', 'X_883', 'X_11', 'X_448', 'X_468', 'X_544', 'X_566', 'X_442', 'X_22', 'X_405', 'X_12', 'X_123', 'X_13', 'X_462', 'X_885', 'X_543', 'X_651', 'X_62', 'X_463', 'X_451', 'X_121', 'X_404', 'X_43', 'X_402', 'X_898', 'X_60', 'X_771', 'X_545', 'X_443', 'X_774', 'X_400', 'X_559', 'X_58', 'X_56', 'X_20', 'X_44', 'X_560', 'X_539', 'X_18', 'X_564', 'X_471', 'X_21', 'X_773', 'X_542', 'X_447', 'X_90', 'X_65', 'X_900', 'X_884', 'X_403', 'X_565', 'X_40', 'X_755', 'X_753', 'X_41', 'X_450', 'X_537', 'X_758', 'X_42', 'X_16', 'X_538', 'X_756', 'X_46', 'X_47']


(LGBMClassifier(random_state=42),
 ['X_1',
  'X_2',
  'X_11',
  'X_12',
  'X_13',
  'X_16',
  'X_18',
  'X_20',
  'X_21',
  'X_22',
  'X_39',
  'X_40',
  'X_41',
  'X_42',
  'X_43',
  'X_44',
  'X_46',
  'X_47',
  'X_48',
  'X_49',
  'X_50',
  'X_51',
  'X_52',
  'X_53',
  'X_54',
  'X_55',
  'X_56',
  'X_57',
  'X_58',
  'X_59',
  'X_60',
  'X_61',
  'X_62',
  'X_63',
  'X_64',
  'X_65',
  'X_66',
  'X_90',
  'X_120',
  'X_121',
  'X_123',
  'X_400',
  'X_401',
  'X_402',
  'X_403',
  'X_404',
  'X_405',
  'X_442',
  'X_443',
  'X_444',
  'X_446',
  'X_447',
  'X_448',
  'X_450',
  'X_451',
  'X_452',
  'X_453',
  'X_454',
  'X_456',
  'X_457',
  'X_458',
  'X_460',
  'X_461',
  'X_462',
  'X_463',
  'X_464',
  'X_468',
  'X_470',
  'X_471',
  'X_536',
  'X_537',
  'X_538',
  'X_539',
  'X_542',
  'X_543',
  'X_544',
  'X_545',
  'X_559',
  'X_560',
  'X_562',
  'X_563',
  'X_564',
  'X_565',
  'X_566',
  'X_651',
  'X_698',
  'X_753',
  'X_754',
  'X_755',
  'X_756',
  'X_757',
  'X_

In [2]:
from sklearn.ensemble import RandomForestClassifier

seperate_code()


def data_modeling_T():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')
    data_T = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]

    lists = []
    for a in make_col()[2]:
        lists.append(int(a[2:]))
    lists.sort()

    cols = []  # 사용할 columns
    for l in lists:
        cols.append(f"X_{l}")

    T_y = data_T["Y_Class"]
    T_X = data_T[cols]

    X_train, X_test, y_train, y_test = train_test_split(T_X, T_y, test_size=0.3, random_state=42)

    model_T = lgb.LGBMClassifier(random_state=42)
    model_T.fit(X_train, y_train)
    prediction = model_T.predict(X_test)

    print(confusion_matrix(prediction, y_test))
    print(f"T_정답률:{accuracy_score(prediction, y_test):.3f}")

    return model_T, cols


data_modeling_T()

NameError: name 'seperate_code' is not defined

In [ ]:
# 이거는 초기 데이터 안지우고 해보는거임...

from sklearn.ensemble import RandomForestClassifier

seperate_code()

def data_modeling_T():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'train.csv')
    data_T = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]

    lists = []
    for a in make_col()[2]:
        lists.append(int(a[2:]))
    lists.sort()

    cols = []  # 사용할 columns
    for l in lists:
        cols.append(f"X_{l}")

    T_y = data_T["Y_Class"]
    T_X = data_T.drop(["Y_Class", "Y_Quality"], axis=1).select_dtypes(exclude=['object'])
    # T_X = data_T[['X_454', 'X_120', 'X_456', 'X_401', 'X_899', 'X_536', 'X_757', 'X_772', 'X_461', 'X_775', 'X_458', 'X_754', 'X_460', 'X_698', 'X_766', 'X_901', 'X_446', 'X_453', 'X_763', 'X_562', 'X_457', 'X_452', 'X_2', 'X_769', 'X_883', 'X_11', 'X_448', 'X_468', 'X_544', 'X_566', 'X_442', 'X_22', 'X_405', 'X_12', 'X_123', 'X_13', 'X_462', 'X_885', 'X_543', 'X_651', 'X_62', 'X_463', 'X_451', 'X_121', 'X_404', 'X_43', 'X_402', 'X_898', 'X_60', 'X_771', 'X_545', 'X_443', 'X_774', 'X_400', 'X_559', 'X_58', 'X_56', 'X_20', 'X_44', 'X_560', 'X_539', 'X_18', 'X_564', 'X_471', 'X_21', 'X_773', 'X_542', 'X_447', 'X_90', 'X_65', 'X_900', 'X_884', 'X_403', 'X_565', 'X_40', 'X_755', 'X_753', 'X_41', 'X_450', 'X_537', 'X_758', 'X_42', 'X_16', 'X_538', 'X_756', 'X_46', 'X_47']]

    X_train, X_test, y_train, y_test = train_test_split(T_X, T_y, test_size=0.3, random_state=42)

    model_T = lgb.LGBMClassifier(random_state=42)
    model_T.fit(X_train, y_train)
    prediction = model_T.predict(X_test)

    #특성중요도 찾기
    feat_labels = X_train.columns
    importances = model_T.feature_importances_

    # argsort : 리스트 인덱스의 정렬됐을 때의 인덱스 값 반환, [::-1] : 뒤집기
    indices = np.argsort(importances)[::-1]
    a = []
    for i in range(X_train.shape[1]):
        if importances[indices[i]] > 1:
            a.append(feat_labels[indices[i]])
            print(feat_labels[indices[i]], )

    print(confusion_matrix(prediction, y_test))
    print(f"T_정답률:{accuracy_score(prediction, y_test):.3f}")
    print(a)


    return model_T, cols

data_modeling_T()

In [48]:
def predict_testset():
    data_path = '/Users/sola/Downloads/open/'
    df = pd.read_csv(data_path + 'test.csv')

    data_A = data_modeling_A()
    data_T = data_modeling_T()

    # A_testset
    df_A = df[df["PRODUCT_CODE"] == "A_31"]

    df_test_A = pd.DataFrame()
    A_cols = data_A[1]

    for c, col in enumerate(A_cols):
        df_test_A[col] = df_A[col]

    # T_testset
    df_T = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")]

    df_test_T = pd.DataFrame()
    T_cols = data_T[1]

    for c, col in enumerate(T_cols):
        df_test_T[col] = df_T[col]

    pred_A = data_A[0].predict(df_test_A)
    pred_T = data_T[0].predict(df_test_T)

    # Code A, T 순서 원래대로 변경
    A_index = df[df["PRODUCT_CODE"] == "A_31"].index
    T_index = df[(df["PRODUCT_CODE"] == "T_31") | (df["PRODUCT_CODE"] == "O_31")].index

    A_add_df = pd.DataFrame()
    A_add_df["index"] = A_index
    A_add_df["predict"] = pred_A

    T_add_df = pd.DataFrame()
    T_add_df["index"] = T_index
    T_add_df["predict"] = pred_T

    predict_data = pd.concat([A_add_df, T_add_df])
    predict_data = predict_data.sort_values(by=["index"], ascending=[True])

    # submit
    output_dir = "Result/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    submit = pd.read_csv(data_path + 'sample_submission.csv')
    submit["Y_Class"] = predict_data["predict"].values
    submit.to_csv(os.path.join(output_dir, "./sola_submission1.csv"), index=False)


In [49]:
seperate_code()
# predict_testset()
predict_testset()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').